# CLIP Guided Stable Diffusion Community Sample
Test edildi. Kisa denemelerde yararli bir fark gorulmedi

In [7]:
#----------------------------------------------------------------------------------------#
# Add diffusers path to sys path
import sys
import os
# Get Diffusers path from environment variable
diffusers_path = os.getenv('DIFFUSERS_PATH')
print(f"Diffusers path: {diffusers_path}")
if diffusers_path is None:
    raise ValueError("Please set DIFFUSERS_PATH environment variable to Diffusers path")
sys.path.append(diffusers_path+"/src")
#----------------------------------------------------------------------------------------#

from diffusers import StableDiffusionPipeline
import torch
models = [r'f:\sd_models\ghostmix_v2',
          r'f:\sd_models\aesteticmix',
          r'f:\sd_models\revAnimated_v122',]
device = "cuda"

# print pytorch version
print(torch.__version__)

Diffusers path: f:\dev\aitnew\diffusers
2.0.1


In [28]:
pipe = StableDiffusionPipeline.from_pretrained(models[1], safety_checker=None)
pipe = pipe.to(device, torch.float16)

prompt = "Flower design pattern with watercolor background (sharp, crisp, masterpiece, top quality, best quality, official art, beautiful and aesthetic:1.2), extreme detailed,colorful,highest detailed:1.3"
prompt2 = "fantasy book cover, full moon, fantasy forest landscape, golden vector elements, fantasy magic, dark light night, intricate, elegant, sharp focus, illustration, highly detailed, digital painting, concept art, matte, art by WLOP and Artgerm and Albert Bierstadt, masterpiece"
negative_prompt = "(worst quality, low quality:2), monochrome, zombie,overexposure, watermark,text,bad anatomy,bad hand,extra hands,extra fingers,too many fingers,fused fingers,bad arm,distorted arm,extra arms,fused arms,extra legs,missing leg,disembodied leg,extra nipples, detached arm, liquid hand,inverted hand,disembodied limb, small breasts, loli, oversized head,extra body,completely nude, extra navel,easynegative,(hair between eyes),sketch, duplicate, ugly, huge eyes, text, logo, worst face, (bad and mutated hands:1.3),  (blurry:2.0), horror, geometry, bad_prompt, (bad hands), (missing fingers), multiple limbs, bad anatomy, (interlocked fingers:1.2), Ugly Fingers, (extra digit and hands and fingers and legs and arms:1.4), ((2girl)), (deformed fingers:1.2), (long fingers:1.2),(bad-artist-anime), bad-artist, bad hand, extra legs ,(ng_deepnegative_v1_75t)"


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [29]:
generator = torch.Generator(device="cuda").manual_seed(2)
image = pipe(
            prompt
            ,num_inference_steps = 50
            ,height = 512
            ,width = 512
            , guidance_scale = 6
            #, negative_prompt = negative_prompt
            , generator = generator
            ).images[0]
# save the image
image.save('./test1_out/normal_image.png')

  0%|          | 0/50 [00:00<?, ?it/s]

In [31]:
from transformers import CLIPImageProcessor, CLIPModel
feature_extractor = CLIPImageProcessor.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K")
clip_model = CLIPModel.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K", torch_dtype=torch.float16)
generator = torch.Generator(device="cuda").manual_seed(2)

guided_pipeline = DiffusionPipeline.from_pretrained(
    models[1],
    custom_pipeline="clip_guided_stable_diffusion",
    clip_model=clip_model,
    feature_extractor=feature_extractor,    
    torch_dtype=torch.float16,
)
guided_pipeline.enable_attention_slicing()
guided_pipeline = guided_pipeline.to("cuda")

image = guided_pipeline(
    prompt,    
    num_inference_steps=50,
    guidance_scale=7.5,
    clip_guidance_scale=6,
    num_cutouts=4,
    use_cutouts=False,
    generator=generator,
).images[0]
image.save("./test1_out/guided_image.png")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]